In [1]:
import gensim
import pandas as pd

C:\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
with open("./data/ts.txt", "rb") as f:
    product_ts = f.readlines()

In [6]:
print(product_ts[0])

196 14084 12427 26088 26405 <EOO> 196 10258 12427 13176 26088 13032 <EOO> 196 12427 10258 25133 30450 <EOO> 196 12427 10258 25133 26405 <EOO> 196 12427 10258 25133 10326 17122 41787 13176 <EOO> 196 12427 10258 25133 <EOO> 196 10258 12427 25133 13032 <EOO> 12427 196 10258 25133 46149 49235 <EOO> 49235 46149 25133 196 10258 12427 <EOO> 196 46149 39657 38928 25133 10258 35951 13032 12427 <EOO> 196 25133 38928 26405 39657 10258 13032 26088 27845 49235 46149



In [3]:
split_product_ts = []
for prd in product_ts:
    split_product_ts.append(prd.split())

In [4]:
print(split_product_ts[1000])

['25745', '39581', '34287', '22565', '36393', '39832', '17743', '43409', '45596', '41473', '24838', '28199', '14778', '4137', '33894', '29432', '9508', '<EOO>', '9508', '45596', '14778', '33894', '3696', '33000', '39832', '31663', '10673', '25344', '4792', '36978', '41650', '43954', '13370', '47307', '<EOO>', '9508', '14778', '33894', '28199', '31663', '4792', '24799', '36978', '41650', '34287', '32650', '20561', '17982', '34023', '49279', '37158', '1609', '<EOO>', '33894', '14778', '28199', '36978', '4792', '17982', '24799', '45596', '39832', '37158', '17743', '20955', '30696', '16786', '35390', '28647', '<EOO>', '9508', '28199', '33894', '7606', '20955', '17339', '30597', '7168', '25266', '31964', '32813', '42976', '17982', '34862']


In [5]:
# create word2vec model
model = gensim.models.Word2Vec(split_product_ts, size=32, window=3, min_count=1, workers=2)

In [9]:
# save in binary format
#gensim.models.Word2Vec.save_word2vec_format(model, './data/prod2vec_32.bin', binary=True)
model.wv.save_word2vec_format('./data/prod2vec_32.vec')

In [13]:
# import products
products = pd.read_csv("./data/products.csv")

In [20]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [10]:
# explore prod2vec model
model.most_similar(positive=['196'], topn=5)
## they are all sodas GREAT!!!
model.most_similar(positive=['27845'], topn=5)
## milk and yoghurt
model.most_similar(positive=['13379'], topn=5)
## noodles and sauce
model.most_similar(positive=['<EOO>'], topn=5)
## EOO: Banas are very similar -> is the most commen product (?)

[('13176', 0.7737448215484619),
 ('49235', 0.7622644305229187),
 ('24852', 0.7421951293945312),
 ('25146', 0.7301254868507385),
 ('27086', 0.6835342645645142)]

In [25]:
prd = '33894'
topn = model.most_similar(positive=[prd], topn=5)
topn_id = [int(p[0]) for p in topn]
topn_id.insert(0, prd)
print(topn_id)

['33894', 25976, 18721, 46448, 47946, 18792]


In [26]:
prods_nbrs = products.loc[products['product_id'].isin(topn_id)]
print(prods_nbrs)

<class 'pandas.core.frame.DataFrame'>
       product_id                                       product_name  \
18720       18721      Baked Rice and Corn Puffs, Aged White Cheddar   
18791       18792              Goldfish Pretzel Baked Snack Crackers   
25975       25976  Pepperidge Farm Colors Cheddar Baked Snack Cra...   
33893       33894  Goldfish Cheddar Baked Snack Crackers Multi Packs   
46447       46448  Pepperidge Farm Goldfish Neon Colors Baked Sna...   
47945       47946                                      Nutella & Go!   

       aisle_id  department_id  
18720       107             19  
18791        78             19  
25975        78             19  
33893        78             19  
46447        78             19  
47945        88             13  


In [49]:
## find product IDs for specific words
products.loc[products['product_name'].str.contains("white wine", case = False)]

,product_id,product_name,aisle_id,department_id
1167,1168,"White Wine Winemaker's Blend California, 2010",62,5
1941,1942,Aged White Wine Vinegar,19,13
3875,3876,Sofia Blanc De Blancs Sparkling White Wine,134,5
5297,5298,30 Minute Marinade Herb & White Wine With Lemo...,5,13
8094,8095,"White Wine Salami, Pinot Grigio",96,20
14286,14287,White Wine Vinegar,19,13
18113,18114,White Wine & Garlic Sauces Skillet Shrimp Scampi,5,13
31051,31052,White Wine Chicken & Couscous,100,21
31661,31662,Organic White Wine Vinegar,19,13
33000,33001,White Wine,62,5


In [54]:
# Woman is to man as king is to ...
#model.most_similar(positive=['woman', 'king'], negative=['man'])

# Kellog's cereal (6213) is to milk (27845) as pasta (33) is to ...
model.most_similar(positive=['6213', '33'], negative=['27845'])

# Bud Light (41816) is to Doritos (2144) as white wine (33001) is to ...
model.most_similar(positive=['33001', '41816'], negative=['2144'])

## doesnt work too well yet...

# How similar are two particular products according to the model?
model.similarity("598", "33001")
## Cabernet Sauvignon and white wine


0.93431654434436595